In [1]:
import requests
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys


import warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome(r"C:\Users\ASUS\chromedriver.exe")

In [3]:
driver.get("https://www.flipkart.com")

In [4]:
# Closing the popup
try:
    driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]').click()
except :
    pass

In [236]:
key = ["Smart watches","Phones","Laptops","Professional Cameras","Headphones","Printers","Monitors","Router","HomeTheatre"]

In [6]:
search_box = driver.find_element_by_xpath('//input[@class="_3704LK"]')

search_btn = driver.find_element_by_xpath('//div[@class="col-12-12 _2oO9oE"]//button')

In [8]:
# To Store the product Urls
urls = []
# Fetching urls of the products
for i in key:
    time.sleep(1)
    try:
        search_box.send_keys(Keys.CONTROL+"a")# Selecting all the text present inside the search box
        search_box.send_keys(Keys.DELETE)# Deleting it
        search_box.send_keys(i)# Sending keys for searching
        search_btn.click()# Clicking on search button
    except StaleElementReferenceException:
        driver.find_element_by_xpath('//input[@class="_3704LK"]').send_keys(Keys.CONTROL+"a")
        driver.find_element_by_xpath('//input[@class="_3704LK"]').send_keys(Keys.DELETE)
        driver.find_element_by_xpath('//div[@class="col-12-12 _2oO9oE"]//button').click()
    
    time.sleep(2)
    for i in driver.find_elements_by_xpath('//a[@class="_1fQZEK" or @class="s1Q9rs"]'):# Extracting product url
        urls.append(i.get_attribute('href'))

In [17]:
Rating = [] # Creating list to store data
review_s = []
full_r = []
for i in urls:
    driver.get(i)
    time.sleep(2)

    # Clicking on all reviews
    try:
        all_review =driver.find_element_by_xpath("//div[@class='_3UAT2v _16PBlm']//span")
        all_review.click()
        time.sleep(2)
    except:
        continue
    # Defining loop to navigate first 10 pages
    for pages in range(0,10):
        
        # Fecthing rating
        try:
            for i in driver.find_elements_by_xpath("//div[@class='col _2wzgFH K0kLPL']/div[1]"):
                if i.text is None:
                    Rating.append("---")
                else:
                    Rating.append(i.text.split("\n")[0])
                
        except StaleElementReferenceException:
            pass  

    
        # Short Reviews
        try:
            for i in driver.find_elements_by_xpath("//div[@class='col _2wzgFH K0kLPL']/div[1]"):
                if i.text is None:
                    review_s.append("---")
                else:
                    review_s.append(i.text.split("\n"))
        except StaleElementReferenceException:
            pass
        
        # Fetching full reviews
        try:
            for i in driver.find_elements_by_xpath('//div[@class="t-ZTKy"]'):
                if i.text is None:
                    full_r.append("---")
                else:
                    full_r.append(i.text)
        except StaleElementReferenceException:
            pass
        
    
        try:
            button=driver.find_element_by_xpath("//*[contains(text(), 'Next')]")
            driver.execute_script("arguments[0].click();", button)
            time.sleep(2)
        
        except:
            break
    

### Fetching data from Amazon.in

In [30]:
driver.get("https://www.amazon.in")

In [271]:
# To Store the product Urls
amzn_urls = []
for i in key:
    time.sleep(1)
    try:
        driver.find_element_by_id('twotabsearchtextbox').clear()
        driver.find_element_by_id('twotabsearchtextbox').send_keys(i)
        time.sleep(1)
        driver.find_element_by_xpath('//span[@class="nav-search-submit-text nav-sprite nav-progressive-attribute"]').click()
        time.sleep(2)
        for j in driver.find_elements_by_xpath('//h2[@class="a-size-mini a-spacing-none a-color-base s-line-clamp-2"]//a'):
            amzn_urls.append(j.get_attribute('href'))  
    except NoSuchElementException:
        driver.refresh()
        continue

In [296]:
for i in amzn_urls:
    driver.get(i)
    time.sleep(2)

    try:
        all_review =driver.find_element_by_xpath('//div[@id="reviews-medley-footer"]//a')
        all_review.click()
        time.sleep(2)
    except:
        continue
    
    try:
            driver.find_element_by_xpath('//a[@data-hook="cr-translate-these-reviews-link"]').click()
    except:
        pass
    
    
        
    try:
        for i in driver.find_elements_by_xpath('//*[@class="a-section celwidget"]/div[2]/a[1]'):
            if i.text is " ":
                Rating.append("---")
            else:
                Rating.append(i.get_attribute('title').split(".")[0])
                
    except StaleElementReferenceException:
        pass  

    
        
    try:
        for i in driver.find_elements_by_xpath('//a[@data-hook="review-title"]'):
            if i.text is " ":
                review_s.append("---")
            else:
                review_s.append(i.text.split("\n"))
    except StaleElementReferenceException:
        pass
    
    try:
        for i in driver.find_elements_by_xpath('//span[contains(@data-hook,"review-body")]'):
            if i.text is " ":
                full_r.append("---")
            else:
                full_r.append(i.text)
    except NoSuchElementException:
        for i in range(0,len(review_s)):
            full_r.append("--")


In [297]:
print(len(Rating),len(full_r),len(review_s))

20067 20067 20067


In [299]:
df = pd.DataFrame({"Ratings":Rating,"Full Reviews":full_r,"Short Reviews":review_s})

In [300]:
df.to_csv("Nlp_project_dataset.csv")

In [301]:
df.head()

,Ratings,Full Reviews,Short Reviews
0,5,"Awesome Watch , More better than I ve expected...","[5, Excellent]"
1,5,It's charging time Is not good If you will use...,"[5, Awesome]"
2,5,Extremely value for money,"[5, Terrific]"
3,5,Nice product,"[5, Best in the market!]"
4,3,Super,"[3, Fair]"
